In [1]:
using Distributions, LinearAlgebra, Statistics, StatsFuns
cd("../src/")
include("hcmvn.jl")
include("hchol.jl")
include("generate.jl")

Σ_1d_generate (generic function with 1 method)

## Settings

In [2]:
iters = 20
# ns = [512, 1024, 2048]
ns = [256, 512, 1024]
ms = [16, 32, 64]
d = 4

4

## 4.2 Simulation with a constant covariance matrix

In [3]:
θ = 0.7

0.7

In [4]:
# true value
function cc_prob(n::Int, θ::T, b::Array{T, 1}; ns::Int = 10000) where T<:AbstractFloat
    # calculate values using MC 
    s = 0
    x = rand(Normal(0, 1), ns)
    for i in 1:ns
        integ = 1
        for j in 1:n
            integ *= normcdf((b[j] + sqrt(θ)*x[i])/(1-θ))
        end
        s += integ
    end
    return s/ns
end

cc_prob (generic function with 1 method)

In [5]:
ans1_m1 = zeros(length(ns), length(ms), iters)
time1_m1 = zeros(length(ns), length(ms))
ans1_m2 = zeros(length(ns), length(ms), iters)
time1_m2 = zeros(length(ns), length(ms))
ans1_m3 = zeros(length(ns), length(ms), iters)
time1_m3 = zeros(length(ns), length(ms))

real = zeros(length(ns), length(ms), iters)

for i in 1:length(ns)
    n = ns[i]
    Σ_const = Σ_const_generate(n, θ)
    for j in 1:length(ms)
        m = ms[j] 
        for p in 1:iters
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            time1_m1[i, j] += @elapsed ans1_m1[i, j, p] += HMVN(a, b, Σ_const, m)
            time1_m2[i, j] += @elapsed ans1_m2[i, j, p] += HCMVN(a, b, Σ_const, d, m)
            time1_m3[i, j] += @elapsed ans1_m3[i, j, p] += HRCMVN(a, b, Σ_const, d, m)
            real[i, j, p] += cc_prob(n, θ, b)
        end
    end
end

In [6]:
mapslices(mean, abs.((ans1_m1 .- real) ./real), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0821885  0.0711093  0.0866144
 0.0894406  0.078757   0.0667966
 0.10583    0.0804581  0.0977669

In [7]:
time1_m1 / iters

3×3 Array{Float64,2}:
  6.2652  12.989    33.5099
 11.7094  25.605    66.2831
 22.4787  50.9509  131.263 

In [8]:
mapslices(mean, abs.((ans1_m2 .- real) ./real), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0837014  0.070794   0.0859982
 0.08914    0.0776718  0.0660967
 0.10583    0.0826166  0.099097 

In [9]:
time1_m2 / iters

3×3 Array{Float64,2}:
 2.21552  1.96381  1.91069
 4.06864  3.81706  3.7708 
 8.12049  7.53078  7.37052

In [10]:
mapslices(mean, abs.((ans1_m3 .- real) ./real), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0850785  0.0710236  0.0869805
 0.0951403  0.0792406  0.0699556
 0.106798   0.0794499  0.0963232

In [11]:
time1_m3 / iters

3×3 Array{Float64,2}:
 2.16931  1.88877  1.82306
 4.01321  3.72687  3.63179
 8.0559   7.42183  7.25157

## 4.3 Simulation with 1D exponential covariance matrix

In [12]:
β = 10

ans2_m1 = zeros(length(ns), length(ms), iters)
time2_m1 = zeros(length(ns), length(ms))
ans2_m2 = zeros(length(ns), length(ms), iters)
time2_m2 = zeros(length(ns), length(ms))
ans2_m3 = zeros(length(ns), length(ms), iters)
time2_m3 = zeros(length(ns), length(ms))

real_1d = zeros(length(ns), length(ms), iters)

for i in 1:length(ns)
    n = ns[i]
    Σ_1d = Σ_1d_generate(n, β)
    L_1d = cholesky(Σ_1d).L
    for j in 1:length(ms)
        m = ms[j] 
        for p in 1:iters
            a = fill(-Inf, n)
            b = rand(Uniform(0, n), n)
            time2_m1[i, j] += @elapsed ans2_m1[i, j, p] += HMVN(a, b, Σ_1d, m)
            time2_m2[i, j] += @elapsed ans2_m2[i, j, p] += HCMVN(a, b, Σ_1d, d, m)
            time2_m3[i, j] += @elapsed ans2_m3[i, j, p] += HRCMVN(a, b, Σ_1d, d, m)
            real_1d[i, j, p] += mvn(L_1d, a, b, ns = 10, N = 1000)
#             real_1d[i, j, p] += mvn(L_1d, a, b, ns = 100, N = 10000)
        end
    end
end

In [13]:
mapslices(mean, abs.((ans2_m1 .- real_1d) ./real_1d), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0286787   3.35736e-5  0.000112699
 0.00067124  0.0130732   3.83937e-6 
 0.0264869   0.00270392  0.00570164 

In [14]:
time2_m1 / iters

3×3 Array{Float64,2}:
  5.86971  13.3511   34.678 
 11.6257   26.2018   68.2408
 23.2674   52.3627  135.158 

In [15]:
mapslices(mean, abs.((ans2_m2 .- real_1d) ./real_1d), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0328171    6.27471e-5  0.00898169 
 0.000727088  0.0130737   0.000135329
 0.0264869    0.00283975  0.00570865 

In [16]:
time2_m2 / iters

3×3 Array{Float64,2}:
 2.04259  1.93647  1.90565
 4.02843  3.73714  3.6629 
 7.86726  7.33141  7.17065

In [17]:
mapslices(mean, abs.((ans2_m3 .- real_1d) ./real_1d), dims = [3])

3×3×1 Array{Float64,3}:
[:, :, 1] =
 0.0473321  0.000887856  0.0211089 
 0.0217473  0.019022     0.00160444
 0.0372486  0.0125284    0.00657814

In [18]:
time2_m3 / iters

3×3 Array{Float64,2}:
 2.03777  1.85493  1.81271
 3.97102  3.66528  3.56411
 7.91013  7.28373  7.1079 

## Save the objects

In [24]:
using JLD2, FileIO

In [22]:
cd("../tables")

In [31]:
@save "table3_1.jld2" ans1_m1 ans1_m2 ans1_m3 time1_m1 time1_m2 time1_m3 iters

In [32]:
@save "table3_2.jld2" ans2_m1 ans2_m2 ans2_m3 time2_m1 time2_m2 time2_m3 iters